In [1]:
import re
import pandas as pd
import gensim
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import en_core_web_sm
import nltk
import os
from nltk.tokenize import word_tokenize
import emoji

D:\deniz\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [32]:
df = pd.read_csv('Data/aftercorona.csv')
df.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,site
0,2.730776e+07,50931,AppelquistP,"Damn I thought I was finally in the #Matrix or #blackmirror or #Aliens landed and this was their way of a soft opening...\nNow, I appreciate #Interns even more.. https://t.co/AjdTC8V6tq",0,OZ,18/06/2021,130,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
1,2.880588e+09,27578,kaarnama13,â€œNosediveâ€ episode from #blackmirror gets closer to reality with kencil culture,0,Aage se right,18/06/2021,669,False,6,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
2,2.370116e+09,115767,Niggvtivity,I've just watched episode S03 | E05 of Black Mirror! #BlackMirror https://t.co/gYm02z0pa4 #tvtime,0,"Bonneuil-sur-Marne, France",18/06/2021,795,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
3,4.817271e+07,4971,DystopicRedhead,"The perfect description of the #Tories and their voters by one of my favourite screenwriters in the whole ðŸŒŽ, the Mastermind behind #BlackMirror, Charlie Brooker \n\n#FBPE #FBPA #FBPPR #ToriesOut \n#ToryCriminals https://t.co/W8buFjRf1b",4,United Kingdom,18/06/2021,2117,False,8,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
4,1.402080e+18,13,smash_media_us,ã€ #ONEUS Ã— #smash. ã€‘\n\nONEUS is on now smash.â£ï¸\n\nNew song #BLACKMIRROR is on smash. exclusive vertical music video!\n\nClick here to watchâ¬‡ï¸\nhttps://t.co/Uh9mE40PCx,0,NaN,18/06/2021,762,False,5,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json


In [34]:
df.columns

Index(['user_id', 'user_statuses', 'name', 'tweet', 'retweets', 'location',
       'created', 'followers', 'is_user_verified', 'favorite_count',
       'in_reply_to_status_id', 'keyword', 'language', 'site'],
      dtype='object')

In [35]:
df.shape

(1735, 14)

In [36]:
def clean_complete_vader(tweet):
    """
    tweet: pandas series
    prepares tweets complete cleaning for further lemmatization and dering embeddings
    """
    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    
    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}
    # tweet = tweet.str.replace(r':-\)', ' smile')
    # tweet = tweet.str.replace(r':-\(', ' sad')
    # tweet = tweet.str.replace(r':-\/', ' confused')
    # tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')
    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')
    tweet = tweet.str.replace('"', '')
    tweet = tweet.str.replace('ℐℓ٥ﻻﻉ√٥υ', '')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #  tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]]

    #remove stop words
    tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]

    return tweet


In [37]:
def lemmatize(tweet):
    '''
    tweet: pandas series
    should be applied on the cleaned tweets to transform words to their initial base form.
    For example: suggests -> suggest, deliveries -> delivery
    '''
    nlp = spacy.load("en_core_web_sm")
    tweet = [nlp(c) for c in tweet]
    tweet = [" ".join([token.lemma_ for token in t]) for t in tweet]
    return tweet

In [38]:
import spacy
df['tweet'] = clean_complete_vader(df.tweet)
df['tweet'] = lemmatize(df.tweet)


In [39]:
df.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,site
0,2.730776e+07,50931,AppelquistP,"Damn think finally # matrix # blackmirror # alien land way soft opening ... now , appreciate # intern more ..",0,OZ,18/06/2021,130,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
1,2.880588e+09,27578,kaarnama13,â€œNosediveâ€ episode # blackmirror get close reality kencil culture,0,Aage se right,18/06/2021,669,False,6,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
2,2.370116e+09,115767,Niggvtivity,I 've watch episode Black Mirror ! # BlackMirror # tvtime,0,"Bonneuil-sur-Marne, France",18/06/2021,795,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
3,4.817271e+07,4971,DystopicRedhead,"the perfect description # tory voter favourite screenwriter ðÿœž , mastermind # BlackMirror , Charlie Brooker # FBPE # FBPA # FBPPR # ToriesOut # ToryCriminals",4,United Kingdom,18/06/2021,2117,False,8,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
4,1.402080e+18,13,smash_media_us,ã€ # ONEUS # smash . ã€‘ONEUS smash.â£ï¸New song # blackmirror smash . exclusive vertical music video!click watchâ¬‡ï¸,0,NaN,18/06/2021,762,False,5,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json


In [40]:
df = df[['user_id','tweet', 'retweets', 'location',
       'created', 'followers', 'keyword', 'language']]

In [43]:
pd.set_option('display.max_colwidth', None)
df.head(1)

,user_id,tweet,retweets,location,created,followers,keyword,language
0,27307763.0,"Damn think finally # matrix # blackmirror # alien land way soft opening ... now , appreciate # intern more ..",0,OZ,18/06/2021,130,#blackmirror,en


In [22]:
# df.to_csv('to_hive_aftercorona.csv', sep=',', header=False)